In [19]:
import h5py
import plotly.graph_objects as go
import numpy as np 
from jetto_mobo.utils import pad_1d, rgb_to_rgba
import plotly

figure = go.Figure()

def add_trace(ensemble_dir, colour, name):
    max_values = []

    for ensemble_member in range(10):
        hdf5_file = f"{ensemble_dir}/{ensemble_member}/bayesopt.hdf5"
        with h5py.File(hdf5_file) as h5:
            optimisation_step = np.arange(h5["/"].attrs["n_completed_bayesopt_steps"] + 1)
            max_value = np.zeros(len(optimisation_step))
            for i, step in enumerate(optimisation_step):
                if i == 0:
                    max_value[:] = np.max(np.nan_to_num(h5["initialisation/value"], 0))
                else:
                    value = np.max(np.nan_to_num(h5[f"bayesopt/{step}/value"], 0))
                    if value > max_value[i-1]:
                        max_value[i:] = value
        max_values.append(max_value)

    max_values = pad_1d(max_values)
    optimisation_step = np.arange(max_values.shape[1])
    mean = np.nanmean(max_values, axis=0)
    sd = np.nanstd(max_values, axis=0)
    #max_ = np.nanmax(max_values, axis=0)
    #min_ = np.nanmin(max_values, axis=0)

    figure.add_traces(
        [
            go.Scatter(
                x=optimisation_step,
                y=mean + 2*sd,
                name=f'{name}: &mu; + 2&#963;',
                mode='lines',
                line_color=rgb_to_rgba(colour, 0.3),
                showlegend=False,
                legendgroup=name,
            ),
            go.Scatter(
                x=optimisation_step,
                y=mean - 2*sd,
                name=f'{name}: &mu; - 2&#963',
                mode='lines',
                fill='tonexty',
                line_color=rgb_to_rgba(colour, 0.3),
                fillcolor=rgb_to_rgba(colour, 0.3),
                showlegend=False,
                legendgroup=name,
            ),
            go.Scatter(
                x=optimisation_step,
                y=mean,
                mode='lines',
                name=f'{name}: &mu;',
                line_color=colour,
                showlegend=False,
                legendgroup=name,
            ),
            go.Scatter(
                x=[optimisation_step[0]],
                y=[mean[0]],
                mode='lines',
                name=name,
                line_color=colour,
                showlegend=True,
                legendgroup=name,
            )
        ]
    )
add_trace("../data/sobo/ga_pl_ensemble", plotly.colors.qualitative.Vivid[0], "Marsden's piecewise linear")
add_trace("../data/sobo/pl_ensemble", plotly.colors.qualitative.Vivid[1], "Piecewise linear")
add_trace("../data/sobo/sog_ensemble", plotly.colors.qualitative.Vivid[2], "Sum of Gaussians")

figure.update_layout(
    template="simple_white",
    showlegend=True
)
figure.update_xaxes(title="Optimisation step")
figure.update_yaxes(title="Maximum objective value achieved")